# Easily export jupyter cells to python module
https://github.com/fastai/course-v3/blob/master/nbs/dl2/notebook2script.py

In [4]:
! python /tf/main/src/scripts/notebook2script.py evaluation.ipynb

Converted evaluation.ipynb to exp/nb_evaluation.py


In [5]:
# imports
from fastai.text import *

In [7]:
sys.path.append("../../")
from src.proc.exp.nb_proc import *
from src.prep.exp.nb_prep import *

In [8]:
#export
# Evaluation metrics for vulnerability detection - Accuracy, Precision, Recall
def eval_vuln(mdl, tst):
    tps, tns, fps, fns = 0, 0, 0, 0
    print("Hello World")
    for inpt, lbl in zip(tst["query"], tst["res"]):
        print(lbl)
        pred = mdl.predict(inpt, 1, temperature=0.75)
        if lbl == "yes":
            if pred == lbl:
                tps += 1
            else: fns += 1
        else:
            if pred == lbl:
                tns += 1
            else: fps += 1
            
    acc   = (tps + tns) / len(tst)
    prec  = tps / (tps + fps)
    recal = tps / (tps + fns)
    
    return acc, prec, recal

In [80]:
# Dependency downloads
import nltk
# required for meteor to perform similarity score, etc by looking for synonyms, antonyms...
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [227]:
# samples
reference1 = 'It is a guide to action that ensures that the military will forever heed Party commands'
reference2 = 'It is the guiding principle which guarantees the military forces always being under the command of the Party'
reference3 = 'It is the practical guide for the army always to heed the directions of the party'
exact_reference = 'It is a guide to action which ensures that the military always obeys the commands of the party'
references = [reference1, reference2, reference3, exact_reference]
hypothesis1 = 'It is a guide to action which ensures that the military always obeys the commands of the party'
hypothesis2 = 'It is to insure the troops forever hearing the activity guidebook that party direct'
hypotheses = [hypothesis1, hypothesis2]
bad_reference = ['this is a cat']
bad_sentence = 'non matching hypothesis'
near_hypotheses = ['Here is cat','This is a dog', 'This is a dog.', 'this is a cat']

## Bleu Score

In [222]:
# !pip install nltk
from typing import List
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

def _eval_bleu(reference_texts: List[str], generated_text: str, weights: List[int]):
    tokenized_references = [tokenizer.tokenize(reference) for reference in reference_texts]
    tokenized_generated_text = tokenizer.tokenize(generated_text)
    return round(sentence_bleu(
        tokenized_references, 
        tokenized_generated_text, 
        weights=weights),
        4)
    
def eval_bleu1(reference_texts: List[str], generated_text: str):
    return _eval_bleu(reference_texts, 
                      generated_text, 
                      weights = (1,0,0,0))
    
def eval_bleu2(reference_texts: List[str], generated_text: str):
    return _eval_bleu(reference_texts, 
                      generated_text, 
                      weights = (0.5,0.5,0,0))

def eval_bleu3(reference_texts: List[str], generated_text: str):
    return _eval_bleu(reference_texts, 
                      generated_text, 
                      weights = (0.33,0.33,0.33,0))

def eval_bleu4(reference_texts: List[str], generated_text: str):
    return _eval_bleu(reference_texts, 
                      generated_text, 
                      weights = (0.25,0.25,0.25,0.25))

In [223]:
# Sample Bleu Score
bleu_ref = ['this is small test']
bleu_cand = 'this is a test'
print(eval_bleu1(bleu_ref, bleu_cand))
print(eval_bleu2(bleu_ref, bleu_cand))
print(eval_bleu3(bleu_ref, bleu_cand))
print(eval_bleu4(bleu_ref, bleu_cand))

0.75
0.5
0.0
0.0


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


## Meteor Score

In [224]:
from nltk.translate.meteor_score import meteor_score
def eval_meteor(reference_texts: List[str], generated_text: str):
    return round(meteor_score(reference_texts, generated_text, preprocess=str.lower), 4)

In [225]:
#good hypotheses
[eval_meteor(references, hypothesis) for hypothesis in hypotheses]

[0.9999, 0.3696]

In [228]:
# bad hypothesis
round(meteor_score(bad_reference, bad_sentence), 4),  eval_meteor(bad_reference, bad_sentence)

(0.0, 0.0)

In [230]:
eval_meteor(bad_reference, 'this is a cat')

0.9922

In [231]:
# near hypotheses
[eval_meteor(bad_reference, near_hypothesis) for near_hypothesis in near_hypotheses]

[0.2564, 0.7361, 0.7361, 0.9922]

# Rouge-L Metric - Incomplete

In [62]:
#setup 
!pip install py-rouge

     |████████████████████████████████| 61kB 2.7MB/s eta 0:00:011
You should consider upgrading via the 'pip install --upgrade pip' command.


In [232]:
import rouge
def _eval_rougeL_single_ref(reference_text: str, generated_text: str):
    evaluator = rouge.Rouge(metrics=['rouge-l'],
                           max_n=4,
#                            limit_length=True,
#                            length_limit=100,
#                            length_limit_type='words',
                           apply_avg=0,
                           apply_best=0,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)
    # scores = evaluator.get_scores(all_hypothesis, all_references)
    # watch out, it takes hypothesis first and then references.
    score = evaluator.get_scores(generated_text, reference_text)['rouge-l'][0]
    score_p = score['p'][0]
    score_f = score['f'][0]
    score_r = score['r'][0]
    return [score_p, score_r, score_f]

def eval_rougeL(reference_texts: List[str], generated_text: str):
    scores = [
        _eval_rougeL_single_ref(
            reference, 
            generated_text) 
        for reference in reference_texts]
#     return scores
    result_df = pd.DataFrame(scores)
    # be extra careful, mislabeling is going to be really damaging.
    result_df.columns=['p', 'r', 'f']
    return result_df
    

In [163]:
hypothesis_1 = "King Norodom Sihanouk has declined requests to chair a summit of Cambodia 's top political leaders , saying the meeting would not bring any progress in deadlocked negotiations to form a government .\nGovernment and opposition parties have asked King Norodom Sihanouk to host a summit meeting after a series of post-election negotiations between the two opposition groups and Hun Sen 's party to form a new government failed .\nHun Sen 's ruling party narrowly won a majority in elections in July , but the opposition _ claiming widespread intimidation and fraud _ has denied Hun Sen the two-thirds vote in parliament required to approve the next government .\n"
references_1 = ["Prospects were dim for resolution of the political crisis in Cambodia in October 1998.\nPrime Minister Hun Sen insisted that talks take place in Cambodia while opposition leaders Ranariddh and Sam Rainsy, fearing arrest at home, wanted them abroad.\nKing Sihanouk declined to chair talks in either place.\nA U.S. House resolution criticized Hun Sen's regime while the opposition tried to cut off his access to loans.\nBut in November the King announced a coalition government with Hun Sen heading the executive and Ranariddh leading the parliament.\nLeft out, Sam Rainsy sought the King's assurance of Hun Sen's promise of safety and freedom for all politicians.",
                    "Cambodian prime minister Hun Sen rejects demands of 2 opposition parties for talks in Beijing after failing to win a 2/3 majority in recent elections.\nSihanouk refuses to host talks in Beijing.\nOpposition parties ask the Asian Development Bank to stop loans to Hun Sen's government.\nCCP defends Hun Sen to the US Senate.\nFUNCINPEC refuses to share the presidency.\nHun Sen and Ranariddh eventually form a coalition at summit convened by Sihanouk.\nHun Sen remains prime minister, Ranariddh is president of the national assembly, and a new senate will be formed.\nOpposition leader Rainsy left out.\nHe seeks strong assurance of safety should he return to Cambodia.\n",
                    ]

hypothesis_2 = "China 's government said Thursday that two prominent dissidents arrested this week are suspected of endangering national security _ the clearest sign yet Chinese leaders plan to quash a would-be opposition party .\nOne leader of a suppressed new political party will be tried on Dec. 17 on a charge of colluding with foreign enemies of China '' to incite the subversion of state power , '' according to court documents given to his wife on Monday .\nWith attorneys locked up , harassed or plain scared , two prominent dissidents will defend themselves against charges of subversion Thursday in China 's highest-profile dissident trials in two years .\n"
references_2 = "Hurricane Mitch, category 5 hurricane, brought widespread death and destruction to Central American.\nEspecially hard hit was Honduras where an estimated 6,076 people lost their lives.\nThe hurricane, which lingered off the coast of Honduras for 3 days before moving off, flooded large areas, destroying crops and property.\nThe U.S. and European Union were joined by Pope John Paul II in a call for money and workers to help the stricken area.\nPresident Clinton sent Tipper Gore, wife of Vice President Gore to the area to deliver much needed supplies to the area, demonstrating U.S. commitment to the recovery of the region.\n"

all_hypothesis = [hypothesis_1, hypothesis_2]
all_references = [references_1, references_2]

In [237]:
score = eval_rougeL(references, hypothesis1)
score

,p,r,f
0,0.663387,0.731803,0.695917
1,0.713275,0.713275,0.713275
2,0.561231,0.619111,0.588752
3,1.000000,1.000000,1.000000


In [238]:
score.mean(axis=0), score.std(axis=0), score.median(axis=0)

(p    0.734473
 r    0.766047
 f    0.749486
 dtype: float64, p    0.187989
 r    0.163586
 f    0.175854
 dtype: float64, p    0.688331
 r    0.722539
 f    0.704596
 dtype: float64)

In [239]:
score.std(axis=0)

p    0.187989
r    0.163586
f    0.175854
dtype: float64